# Azure AI Foundry 워크샵 환경 설정

이 노트북은 Azure AI Foundry 워크샵을 위한 환경 설정 방법을 안내합니다.

## 전제 조건
- Python 3.8 이상
- Azure AI 서비스에 접근 가능한 Azure 구독
- 파이썬 기본 지식

In [ ]:
# Install required packages (if not already installed):
!pip install azure-identity azure-ai-projects

## Azure 인증 설정
먼저 Azure 자격 증명 및 설정을 확인합니다.

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
import os

# Initialize Azure credentials
try:
    credential = DefaultAzureCredential()
    print("✓ Successfully initialized DefaultAzureCredential")
except Exception as e:
    print(f"× Error initializing credentials: {str(e)}")

## AI 프로젝트 클라이언트 초기화

> **참고:** 계속 진행하기 전에 다음 사항을 확인하세요:
> 1. `.env.local` 파일을 `.env`에 복사합니다.
> 2. `.env` 파일에서 프로젝트 연결 문자열을 업데이트합니다.
> 3. Azure AI Foundry에 허브 및 프로젝트를 프로비전하여 배포해서 가지고 있어야 합니다.

프로젝트의 설정 아래에서 [Azure AI Foundry](https://ai.azure.com) 프로젝트 연결 문자열을 찾을 수 있습니다:

<img src="proj-conn-string.png" alt="Project Connection String Location" width="600"/>



## AIProjectClient 이해하기

AIProjectClient는 Azure AI 서비스와의 상호 작용을 위한 핵심 구성 요소입니다:

- **연결 관리**: OpenAI 모델과 같은 Azure AI 리소스를 나열하고 접근합니다.
- **인증 처리**: Azure 자격 증명을 사용하여 안전하게 연결합니다.
- **모델 접근 활성화**: AI 모델 및 배포를 사용하기 위한 인터페이스를 제공합니다.
- **프로젝트 설정 관리**: Azure AI 프로젝트의 구성을 제어합니다.

클라이언트에서 다음이 필요합니다:
- 프로젝트 연결 문자열 (Azure AI project 설정에서 찾을 수 있습니다)
- 유효한 자격증명

프로젝트 연결 문자열은 Azure AI Studio의 프로젝트 설정에서 찾을 수 있습니다:



In [ ]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Initialize AIProjectClient with connection string and credentials
try:
    client = AIProjectClient.from_connection_string(
        conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

## 모델에 대한 접근과 연결 검증
[사전 요구 사항](../README.md#-prerequisites)에 지정된 모든 필수 모델 및 연결에 액세스할 수 있는지 확인해 보겠습니다.

유효성을 검사해봅시다:
- 채팅/완성을 위한 GPT 모델(gpt-4o, gpt-4o-mini) 
- 벡터 검색을 위한 임베딩 모델 
- [Bing으로 Grounding](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?view=azure-python-preview&tabs=python&pivots=overview)
- Azure AI Search 연결

이 유효성 검사를 통해 AI 애플리케이션을 빌드하는 데 필요한 모든 구성 요소를 갖출 수 있습니다.

In [ ]:
# List the properties of all connections
connections = client.connections.list()
print(f"====> Listing of all connections (found {len(connections)}):")
for connection in connections:
    print(connection)

# List the properties of all connections of a particular "type" (in this sample, Azure OpenAI connections)
connections = client.connections.list(
    connection_type=ConnectionType.AZURE_OPEN_AI,
)
print(f"====> Listing of all Azure Open AI connections (found {len(connections)}):")
for connection in connections:
    print(connection)

# Get the properties of the default connection of a particular "type", with credentials
connection = client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SERVICES,
    include_credentials=True,  # Optional. Defaults to "False"
)
print("====> Get default Azure AI Services connection:")
print(connection)

print("====> Get connection by name:")
print(connection)

## 모델 및 검색 연결 유효성 검사
다음 코드로 프로비저닝 및 연결이 올바르게 이루어졌는지 확인합니다:
1. Azure OpenAI 연결을 통한 Azure OpenAI 모델
2. Azure AI 검색 연결을 통한 Azure AI 검색

이 두 서비스는 모두 AI 애플리케이션을 구축하는 데 필수적입니다. OpenAI 모델은 핵심 언어 기능을 제공하는 반면 Azure AI Search는 효율적인 정보 검색 및 지식 기반 기능을 지원합니다.



In [ ]:
# List all connections and check for specific types
conn_list = client.connections.list()
search_conn_id = ""
openai_conn_id = ""

for conn in conn_list:
    conn_type = str(conn.connection_type).split('.')[-1]  # Get the part after the dot
    if conn_type == "AZURE_AI_SEARCH":
        search_conn_id = conn.id
    elif conn_type == "AZURE_OPEN_AI":
        openai_conn_id = conn.id

print(f"\n====> Connection IDs found:")
if not search_conn_id:
    print("Azure AI Search: Not found - Please create an Azure AI Search connection")
else:
    print(f"Azure AI Search: {search_conn_id}")
    
if not openai_conn_id:
    print("Azure OpenAI: Not found - Please create an Azure OpenAI connection") 
else:
    print(f"Azure OpenAI: {openai_conn_id}")